<a href="https://colab.research.google.com/github/hamidreza2015/pythorch_experiences/blob/master/Cifar10_Classification_VGG11_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

from torch.utils.tensorboard import SummaryWriter

In [15]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])



# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, 
            download=True, transform=train_transform)

test_dataset = datasets.CIFAR10(root='./data', train=False, 
            download=True, transform=valid_transform)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
batch_size = 64

valid_size=0.1

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))


np.random.seed(42)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)


train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=batch_size, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=batch_size, sampler=valid_sampler)


In [0]:
writer = SummaryWriter()

In [0]:
images ,labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

writer.add_image('images' , grid)


In [0]:
import math

import torch.nn as nn
import torch.nn.init as init

__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]


class VGG(nn.Module):
    '''
    VGG model 
    '''
    def __init__(self, features):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10),
        )
         # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 
          512, 512, 512, 512, 'M'],
}


def vgg11():
    """VGG 11-layer model (configuration "A")"""
    return VGG(make_layers(cfg['A']))


def vgg11_bn():
    """VGG 11-layer model (configuration "A") with batch normalization"""
    return VGG(make_layers(cfg['A'], batch_norm=True))


def vgg13():
    """VGG 13-layer model (configuration "B")"""
    return VGG(make_layers(cfg['B']))


def vgg13_bn():
    """VGG 13-layer model (configuration "B") with batch normalization"""
    return VGG(make_layers(cfg['B'], batch_norm=True))


def vgg16():
    """VGG 16-layer model (configuration "D")"""
    return VGG(make_layers(cfg['D']))


def vgg16_bn():
    """VGG 16-layer model (configuration "D") with batch normalization"""
    return VGG(make_layers(cfg['D'], batch_norm=True))


def vgg19():
    """VGG 19-layer model (configuration "E")"""
    return VGG(make_layers(cfg['E']))


def vgg19_bn():
    """VGG 19-layer model (configuration 'E') with batch normalization"""
    return VGG(make_layers(cfg['E'], batch_norm=True))

In [7]:
# create a complete CNN
model = vgg11()
print(model)

writer.add_graph(model ,input_to_model=images)

# move tensors to GPU if CUDA is available
if torch.cuda.is_available():
    model.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kern

In [0]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=9, gamma=0.9)

In [0]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [10]:
# number of epochs to train the model
n_epochs = 36 # you may increase this number to train a final model

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    total_correct = 0.0
    total_loss = 0.0
    
    ###################
    # train the model #
    ###################
    scheduler.step()
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        total_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
        
        total_correct += get_num_correct(output, target)
    
    # calculate average losses
    train_loss = total_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
       
    
    writer.add_scalar('Loss',total_loss, epoch)
    writer.add_scalar('Number Correct',total_correct, epoch)
    writer.add_scalar('Accuracy', (total_correct/(len(train_dataset)*0.1)) , epoch)
    
    
    
    # print training/validation statistics 
    print('Epoch: {} Total_Loss: {:.6f} Training_Loss: {:.6f} Validation_Loss: {:.6f} Total_Correct: {}'.format(
        epoch, total_loss ,train_loss, valid_loss ,total_correct))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

Epoch: 1 Total_Loss: 90919.046750 Training_Loss: 1.818381 Validation_Loss: 0.180438 Total_Correct: 1462.0
Validation loss decreased (inf --> 0.180438).  Saving model ...
Epoch: 2 Total_Loss: 71364.685709 Training_Loss: 1.427294 Validation_Loss: 0.142726 Total_Correct: 2355.0
Validation loss decreased (0.180438 --> 0.142726).  Saving model ...
Epoch: 3 Total_Loss: 59453.760228 Training_Loss: 1.189075 Validation_Loss: 0.125552 Total_Correct: 2660.0
Validation loss decreased (0.142726 --> 0.125552).  Saving model ...
Epoch: 4 Total_Loss: 50140.912100 Training_Loss: 1.002818 Validation_Loss: 0.101283 Total_Correct: 3255.0
Validation loss decreased (0.125552 --> 0.101283).  Saving model ...
Epoch: 5 Total_Loss: 43990.569581 Training_Loss: 0.879811 Validation_Loss: 0.093109 Total_Correct: 3398.0
Validation loss decreased (0.101283 --> 0.093109).  Saving model ...
Epoch: 6 Total_Loss: 39761.426718 Training_Loss: 0.795229 Validation_Loss: 0.081860 Total_Correct: 3560.0
Validation loss decrease

In [37]:
(total_correct/(len(train_dataset)*0.1))

0.6428

In [0]:
test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=64, 
                                          shuffle=True)

In [12]:
model.load_state_dict(torch.load('model_cifar.pt'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
classDict = {'plane':0, 'car':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}
class_selected_ = list(classDict.keys())

In [17]:
test_loss = 0.0 
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
for data , target in test_loader:
    
    if torch.cuda.is_available():
        data , target = data.cuda() , target.cuda()
        
    output = model(data)
    
    loss = criterion(output , target)
    #print(loss)
    
    test_loss += loss.item()*data.size(0)
    #print(test_loss)
    
    _, pred = torch.max(output, 1) 
    
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
    
    for i in range(len(correct)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))


for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            class_selected_[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.448277

Test Accuracy of plane: 90% (907/1000)
Test Accuracy of   car: 97% (970/1000)
Test Accuracy of  bird: 80% (807/1000)
Test Accuracy of   cat: 70% (706/1000)
Test Accuracy of  deer: 84% (845/1000)
Test Accuracy of   dog: 78% (785/1000)
Test Accuracy of  frog: 85% (855/1000)
Test Accuracy of horse: 92% (925/1000)
Test Accuracy of  ship: 91% (913/1000)
Test Accuracy of truck: 88% (886/1000)

Test Accuracy (Overall): 85% (8599/10000)
